## Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

In [59]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




In [28]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x74e7c9516c90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x74e7c9516660>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Hamza and I am a Full Stack Developer")])

AIMessage(content="Hello Hamza, it's nice to meet you!\n\nThat's great that you're a Full Stack Developer. It's a challenging and rewarding field. \n\nWhat kind of projects are you working on these days? Are there any technologies you're particularly passionate about? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 22, 'total_tokens': 84, 'completion_time': 0.112727273, 'prompt_time': 0.001335559, 'queue_time': 0.187866301, 'total_time': 0.114062832}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--20c903f4-1899-4b3b-a578-035670dca3c1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 62, 'total_tokens': 84})

In [30]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Hamza and I am a Full Stack Developer"),
        AIMessage(content="Hello Hamza! It's nice to meet you. \n\nAs a Full Stack Developer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Hamza, and you are a Full Stack Developer!  \n\nIt seems like you've already told me that,  😊  \n\nIs there anything else you'd like to talk about? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 97, 'total_tokens': 142, 'completion_time': 0.081818182, 'prompt_time': 0.002669299, 'queue_time': 0.189236407, 'total_time': 0.084487481}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--384e5118-a6ca-4a6d-9e21-a9f09baa7ee2-0', usage_metadata={'input_tokens': 97, 'output_tokens': 45, 'total_tokens': 142})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [31]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [32]:
config={"configurable":{"session_id":"chat1"}}

In [33]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Hamza and I am a Full Stack Developer")],
    config=config
)

In [34]:
response.content

"Hi Hamza, it's nice to meet you! \n\nThat's awesome that you're a full stack developer.  What kind of projects are you working on these days?  \n\nIs there anything I can help you with? Perhaps you have a coding question, need help brainstorming an idea, or just want to chat about the latest tech trends? 😊  \n\n"

In [35]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Hamza!  😊 \n\nI remembered from our introduction.  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 114, 'total_tokens': 143, 'completion_time': 0.052727273, 'prompt_time': 0.00328342, 'queue_time': 0.188016836, 'total_time': 0.056010693}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7b67aae4-278a-4845-a32b-8ee2553424e4-0', usage_metadata={'input_tokens': 114, 'output_tokens': 29, 'total_tokens': 143})

In [36]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [37]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. \n\nWhat can I do for you today? 🙂\n"

In [38]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nWe met earlier, remember? 😊 \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [39]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [40]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Hamza")]})

AIMessage(content="Hello Hamza, it's nice to meet you!\n\nI'm happy to help answer any questions you have to the best of my ability.  What can I do for you today?  😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 31, 'total_tokens': 77, 'completion_time': 0.083636364, 'prompt_time': 0.00161732, 'queue_time': 0.191387688, 'total_time': 0.085253684}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--391d87cb-4abc-4552-968d-2fac7da3edb1-0', usage_metadata={'input_tokens': 31, 'output_tokens': 46, 'total_tokens': 77})

In [41]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [42]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Hamza")],
    config=config
)

response

AIMessage(content="Hello Hamza! It's nice to meet you.  \n\nI'm happy to help with any questions you have.  Just ask! 😊 \n\nWhat can I do for you today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.00149643, 'queue_time': 0.188883844, 'total_time': 0.083314612}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a1e53cd3-45e7-448a-ae20-90c7bc7aa95c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [43]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Hamza, you told me earlier! 😊 \n\n\n'

In [44]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [45]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Hamza")],"language":"Urdu"})
response.content

'السلام عليكم Hamza!  \n\nمیرا نام ہے Gemma، آپ کی مدد کرنے والی ایک دستی ہے۔ کیا میں آپ کے لیے کچھ کر سکتا ہوں؟  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [46]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [47]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Hamza")],"language":"Urdu"},
    config=config
)
repsonse.content

'سلام Hamza! \n\nخوش آمدی! میں آپ کی مدد کرنے کے لیے تیار ہوں۔ \n\nکیا میں آپ کے لیے کچھ کر سکتا ہوں؟\n'

In [48]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Urdu"},
    config=config,
)

In [49]:
response.content

'آپ کا نام Hamza ہے.  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [50]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/home/hamzabhatti18/Desktop/GenAi/LCEL/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [51]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What is my name?")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [54]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did I ask?")],
        "language": "English",
    }
)
response.content

'You asked "What is 2 + 2?"  \n\n\n\n\nLet me know if you have any other questions!\n'

In [55]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [56]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  Could you please tell me the math problem you'd like me to help you with? 😊 \n\n"